# Goals of this notebook

1. Named Entity Recognition of the summa
2. Build Ontology of the summa


In [1]:
import json
with open("aquinas.json", "r") as handle:
    data = json.load(handle)
    
data[:5]

[{'volume': 'Volume 1',
  'questionTitle': 'Question 1. The nature and extent of sacred doctrine',
  'question': 'Question 1.',
  'articleTitle': 'Article 1. Whether, besides philosophy, any further doctrine is required?',
  'article': 'Article 1.',
  'articleObjection': ['Objection 1. It seems that, besides philosophical science, we have no need of any further knowledge. For man should not seek to know what is above reason: "Seek not the things that are too high for thee" (Sirach 3:22). But whatever is not above reason is fully treated of in philosophical science. Therefore any other knowledge besides philosophical science is superfluous.',
   'Objection 2. Further, knowledge can be concerned only with being, for nothing can be known, save what is true; and all that is, is true. But everything that is, is treated of in philosophical science—even God Himself; so that there is a part of philosophy called theology, or the divine science, as Aristotle has proved (Metaph. vi). Therefore, b

In [2]:
corpus = ["\n".join(i for i in chunk["articleBody"]) for chunk in data]
corpus[0:3]

['On the contrary, It is written (2 Timothy 3:16): "All Scripture, inspired of God is profitable to teach, to reprove, to correct, to instruct in justice." Now Scripture, inspired of God, is no part of philosophical science, which has been built up by human reason. Therefore it is useful that besides philosophical science, there should be other knowledge, i.e. inspired of God.\nI answer that, It was necessary for man\'s salvation that there should be a knowledge revealed by God besides philosophical science built up by human reason. Firstly, indeed, because man is directed to God, as to an end that surpasses the grasp of his reason: "The eye hath not seen, O God, besides Thee, what things Thou hast prepared for them that wait for Thee" (Isaiah 64:4). But the end must first be known by men who are to direct their thoughts and actions to the end. Hence it was necessary for the salvation of man that certain truths which exceed human reason should be made known to him by divine revelation.

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

for i, c in enumerate(corpus[1:]):
    doc = nlp(c)
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.is_stop)
    break


On on ADP IN prep True
the the DET DT det True
contrary contrary NOUN NN pobj False
, , PUNCT , punct False
Augustine Augustine PROPN NNP nsubj False
says say VERB VBZ ROOT False
( ( PUNCT -LRB- punct False
De De PROPN NNP compound False
Trin Trin PROPN NNP nsubj False
. . PUNCT . punct False
xiv xiv PROPN NNP nsubj False
, , PUNCT , punct False
1 1 NUM CD nummod False
) ) PUNCT -RRB- punct False
" " PUNCT '' punct False
to to ADP IN prep True
this this DET DT det True
science science NOUN NN pobj False
alone alone ADV RB advmod True
belongs belong VERB VBZ ROOT False
that that SCONJ IN mark True
whereby whereby ADV WRB advmod True
saving save VERB VBG nsubjpass False
faith faith NOUN NN dobj False
is be AUX VBZ auxpass True
begotten begotten PROPN NNP ccomp False
, , PUNCT , punct False
nourished nourish VERB VBD appos False
, , PUNCT , punct False
protected protect VERB VBN dep False
and and CCONJ CC cc True
strengthened strengthen VERB VBN conj False
. . PUNCT . punct False
" " PUNC

In [68]:
from tqdm import tqdm

class Collector:
    def __self__(self):
        pass
    
    @classmethod
    def collect_logical_units(self, corpus, nlp): 
        self.logical_unit_corpus = []
        for i in tqdm(range(len(corpus)), position=0, leave=True, desc="Collecting Logical Units", ncols=100, bar_format='{l_bar}{bar}|'):
            c = corpus[i]
            doc = nlp(c)
            # Collect logical units
            logical_units = []
            for token in doc:
                # Catch subjects, objects, nouns, and proper nouns
                if "subj" in token.dep_ or "obj" in token.dep_ or "VERB" == token.pos_:
                    #"NOUN" == token.pos_ or
                    #"PROPN" == token.pos_ or
                    #"ROOT" == token.dep_ or
                    if not token.is_stop:
                        logical_units.append([token.text, (token.dep_, token.pos_)])
            self.logical_unit_corpus.append(logical_units)
        return self.logical_unit_corpus
    
    @classmethod
    def collect_logical_entities(self, corpus, nlp):
        self.logical_entity_corpus = []
        for i in tqdm(range(len(corpus)), position=0, leave=True, desc="Collecting Logical Entities", ncols=100, bar_format='{l_bar}{bar}|'):
            c = corpus[i]
            
            doc = nlp(c)

            # Collect logical entities
            logical_entities = []
            for ent in doc.ents:
                logical_entities.append([ent, ent.label_])
            self.logical_entity_corpus.append(logical_entities)
        return self.logical_entity_corpus
    
"""
Rules. 
1. subj - Subject
2. pobj - Proper Object of the Subject
3. 'VERB' - Description of things the subject can do upon objects of the subject
"""
import copy

class Subject:
    def __init__(self, name, objects = [], predicates = []):
        print("[Creating Subject {}]".format(name))
        self.name = name
        self.objects = objects
        self.predicates = predicates

    def addObject(self, newObject):
        print("[Adding Object {}]".format(newObject))
        self.objects.append(newObject)
        
    def addPredicate(self, newPredicate):
        print("[Adding Predicate {}]".format(newPredicate))
        self.predicates.append(newPredicate)
        
    def objectCount(self):
        return len(self.objects)
    
    def predicateCount(self):
        return len(self.predicates)
    
lunits = Collector().collect_logical_units(corpus, nlp)
#lents = Collector().collect_logical_entities(corpus, nlp)

In [85]:
"""
Rules. 
1. subj - Subject
2. pobj - Proper Object of the Subject
3. 'VERB' - Description of things the subject can do upon objects of the subject
"""
import copy

class Subject:
    def __init__(self, name, objects = [], predicates = []):
        print("[Creating Subject {}]".format(name))
        self.name = name
        self.objects = objects
        self.predicates = predicates
        
    def __str__(self):
        return self.name

    def addObject(self, newObject):
        print("[Adding Object {}]".format(newObject))
        self.objects.append(newObject)
        
    def addPredicate(self, newPredicate):
        print("[Adding Predicate {}]".format(newPredicate))
        self.predicates.append(newPredicate)
        
    def objectCount(self):
        return len(self.objects)
    
    def predicateCount(self):
        return len(self.predicates)

for unit in lunits[1:2]:
    subjects = []
    currentSubject = None
    for row in unit:
        
        if "nsubj" == row[1][0]:
            print("___")
            
            if currentSubject:
                #
                # once you're done processing the previous subjects stuff ... (assuming a subject exists, that is)
                # ask yourself two questions:
                #    1. Do I have at least one object?
                #    2. Do I have at least one predicate?
                # If either are false, then insert your stuff into the next subject ahead of their processing.
                #
                print(currentSubject.objectCount(), currentSubject.predicateCount())
                if (currentSubject.objectCount() < 1 or currentSubject.predicateCount() < 1):
                    # continue business as usual
                    subject = Subject(currentSubject.name + " " + row[0], currentSubject.objects, currentSubject.predicates)
                    del subjects[-1]
                else:
                    subject = Subject(row[0])
                    # pass on our stuff into the upcoming subject, this might make more sense
            else:
                subject = Subject(row[0])
            
            currentSubject = copy.deepcopy(subject)
            subjects.append(currentSubject)

        if currentSubject:
            # Predicate
            if row[1][1] == "VERB":
                currentSubject.addPredicate(row[0])
            # Object
            elif "obj" in row[1][0]:
                currentSubject.addObject(row[0])

            # 1. Make a new Subject
            # 2. Begin collecting all of the objects of that subject
            # 3. Begin collecting all predicates of the subject

            #print(row)
        i += 1
    
[i.name for i in subjects]

___
[Creating Subject Augustine]
[Adding Predicate says]
___
0 1
[Creating Subject Augustine Trin]
___
0 1
[Creating Subject Augustine Trin xiv]
[Adding Object science]
[Adding Predicate belongs]
[Adding Predicate saving]
[Adding Object faith]
[Adding Predicate nourished]
[Adding Predicate protected]
[Adding Predicate strengthened]
[Adding Predicate said]
[Adding Object science]
[Adding Object doctrine]
___
4 7
[Creating Subject doctrine]
[Adding Predicate answer]
___
0 1
[Creating Subject doctrine doctrine]
[Adding Predicate bear]
[Adding Object mind]
[Adding Object sciences]
[Adding Predicate proceed]
[Adding Object principle]
[Adding Predicate known]
[Adding Object light]
[Adding Object intelligence]
[Adding Object arithmetic]
[Adding Predicate proceed]
[Adding Object principles]
[Adding Predicate known]
[Adding Object light]
[Adding Object science]
[Adding Object proceeds]
[Adding Object principles]
[Adding Predicate established]
[Adding Object geometry]
[Adding Object principles]


['Augustine Trin xiv',
 'doctrine doctrine',
 'doctrine',
 'musician',
 'principles']